In [1]:
!pip install cassandra-driver
!pip install cassandra-csv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 63.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for cassandra-csv: filename=cassandra_csv-0.3-py3-none-any.whl size=2422 sha256=a662257f03f420ac6e4da8a428e3dfc4f067d5c5d1e9a4fab0dc17944cb3a39e
  Stored in directory: /root/.cache/pip/wheels/b0/20/ad/858c9e56091677a6b81bcb95fa3739f2e53f480404f07e4da0
Successfully built cassandra-csv


In [2]:
import numpy as np
import pandas as pd
import os
import json

In [3]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
def connect_database():
    file_path= os.path.join(os.getcwd(),'mushrooms_token_datastax_cassandra.json')
    with open(file_path) as f:
        file= json.load(f)
        id= file["clientId"]
        secret= file["secret"]

    try: 
        cloud_config= {
        'secure_connect_bundle': os.path.join(os.getcwd(), 'secure-connect-mushrooms.zip')
                    }
        auth_provider = PlainTextAuthProvider(id, secret)
        cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
        session= cluster.connect('ineuron_mlprojects')

        data = session.execute("SELECT * FROM mushroom_csv;")
        return data
    except Exception as e:
      raise e

In [155]:
df = pd.DataFrame([d for d in connect_database()])
df = df.iloc[:-1 , :]
df["stalk_root"]= df["stalk_root"].replace('?', np.nan)
df.head()

ERROR:cassandra.connection:Closing connection <AsyncoreConnection(140358633208560) 78b4b0b2-133e-4455-a9d2-567770f3aef7-asia-south1.db.astra.datastax.com:29042:8ea91611-c9ec-4a7b-8574-ce4856e9239d> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


,classs,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,p,b,f,n,f,n,f,c,n,w,...,y,w,n,p,w,o,e,w,v,d\n
1,p,b,f,n,f,n,f,c,n,w,...,y,w,y,p,w,o,e,w,v,d\n
2,p,b,f,y,f,n,f,c,n,w,...,y,w,n,p,w,o,e,w,v,d\n
3,p,b,f,y,f,n,f,c,n,w,...,y,w,y,p,w,o,e,w,v,d\n
4,p,b,g,w,t,n,f,w,n,w,...,s,w,w,p,w,o,p,w,c,l\n


In [156]:
from sklearn.model_selection import train_test_split
train_set,test_set=train_test_split(df,test_size=0.20,random_state=42)

In [157]:
def target_encode(df):
    try:
        target_map= {
            "p": 0, 
            "e": 1
            }
        df.loc[:,'classs'] = df['classs'].map(target_map)
        return df
    except Exception as e:
        raise e

train_set=target_encode(train_set)
test_set=target_encode(test_set)
train_set

<ipython-input-157-b6fb3fd36ae7>:7: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,'classs'] = df['classs'].map(target_map)
<ipython-input-157-b6fb3fd36ae7>:7: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,'classs'] = df['classs'].map(target_map)


,classs,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
4634,1,f,f,g,t,n,f,c,b,u,...,s,p,g,p,w,o,p,n,v,d\n
6652,1,x,f,n,f,n,f,w,n,w,...,f,w,n,p,w,o,e,w,v,l\n
6140,1,k,y,p,t,n,f,c,b,w,...,s,e,e,p,w,t,e,w,c,w\n
7151,1,x,s,p,t,n,f,c,b,w,...,s,w,w,p,w,t,e,w,c,w\n
438,0,f,s,e,f,f,f,c,n,b,...,k,p,w,p,w,o,e,w,v,d\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5226,1,f,s,w,f,n,f,w,b,h,...,f,w,w,p,w,o,e,k,a,g\n
5390,1,f,y,e,t,n,f,c,b,p,...,s,p,w,p,w,o,p,n,v,d\n
860,0,f,s,w,t,n,f,c,b,r,...,s,w,w,p,w,t,p,r,v,m\n
7603,1,x,y,g,t,n,f,c,b,p,...,s,g,p,p,w,o,p,n,y,d\n


In [158]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

def get_data_transformation_object():
    try:

        categorical_cols = ['cap_surface', 'bruises', 'gill_spacing', 'gill_size', 'gill_color',
    'stalk_root', 'stalk_surface_above_ring', 'stalk_surface_below_ring',
    'ring_type', 'spore_print_color', 'population', 'habitat'] # keeping only the important columns for our model


        # Categorigal Pipeline
        cat_pipeline=Pipeline(
            steps=[
            ('imputer',SimpleImputer(strategy='most_frequent')),
            ('onehotencoder',OneHotEncoder(handle_unknown='ignore', drop= 'first', sparse_output= False))
            #('get_dummies',pd.get_dummies(columns=[categorical_cols], drop_first=True))
            # no need to standardize after get dummies
            ]

        )
        # combine target and catagorical pipeline
        preprocessor=ColumnTransformer([
        #('tar_pipeline', tar_pipeline, target_col),
        ('cat_pipeline',cat_pipeline,categorical_cols)
        ], remainder ='passthrough', n_jobs=-1)
        return preprocessor
        
        

    except Exception as e:
        raise e

In [159]:
# Reading train and test data
train_df = train_set
test_df = test_set
#target_map_obj = self.target_encode()
preprocessing_obj =get_data_transformation_object()

# feature engineering

#features_train = array(train_df.columns)
#features_test = array(test_df.columns)
target_column_name = 'classs'
drop_columns = [target_column_name, 'veil_type', 'cap_shape', 'cap_color', 'odor', 'gill_attachment', 
                'stalk_shape', 'stalk_color_above_ring', 'stalk_color_below_ring', 'ring_number', 'veil_color']

input_feature_train_df = train_df.drop(columns=drop_columns,axis=1)
target_feature_train_df=train_df[target_column_name]

input_feature_test_df=test_df.drop(columns=drop_columns,axis=1)
target_feature_test_df=test_df[target_column_name]

      
          

## Trnasformating using preprocessor obj
input_feature_train_arr=preprocessing_obj.fit_transform(input_feature_train_df)
input_feature_test_arr=preprocessing_obj.transform(input_feature_test_df)


# convert the train & test df to numpy array for fast calculation
#train_arr = np.c_[input_feature_train_arr, np.concatenate([np.array(target_feature_train_df)], axis=1)] # np.c is concatenation
#train_arr = pd.concat([pd.DataFrame(input_feature_train_arr), pd.DataFrame(target_feature_train_df)], axis=1)
#test_arr = pd.concat([pd.DataFrame(input_feature_test_arr), pd.DataFrame(target_feature_test_df)], axis=1)
#test_arr = np.c_[input_feature_test_arr, np.array(target_feature_test_df)]
#train_arr= train_arr.to_numpy() # converting dataframe to numpy array
#test_arr= test_arr.to_numpy()

# saving the pickle files

#train_arr
#test_arr


In [160]:
input_feature_train_arr

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 1.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [161]:
pd.DataFrame(input_feature_train_arr)

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6493,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6494,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6495,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6496,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [162]:
pd.DataFrame(target_feature_train_df)

,classs
4634,1
6652,1
6140,1
7151,1
438,0
...,...
5226,1
5390,1
860,0
7603,1


In [163]:
pd.DataFrame(target_feature_train_df).reset_index().drop(['index'], axis=1)

,classs
0,1
1,1
2,1
3,1
4,0
...,...
6493,1
6494,1
6495,0
6496,1


In [164]:
train_arr = pd.concat([pd.DataFrame(input_feature_train_arr), pd.DataFrame(target_feature_train_df).reset_index().drop(['index'], axis=1)], axis=1, ignore_index= True, join= 'inner')

test_arr = pd.concat([pd.DataFrame(input_feature_test_arr), pd.DataFrame(target_feature_test_df).reset_index().drop(['index'], axis=1)], axis=1, ignore_index= True, join= 'inner')
train_arr

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
3,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
4,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6493,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1
6494,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
6495,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
6496,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [165]:

train_arr= train_arr.to_numpy()
test_arr= test_arr.to_numpy()
test_arr


array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

In [56]:
'''
train_arr = np.c_[input_feature_train_arr, np.array(pd.DataFrame(target_feature_train_df).reset_index().drop(['index'], axis=1))]
test_arr = np.c_[input_feature_test_arr, np.array(pd.DataFrame(target_feature_test_df).reset_index().drop(['index'], axis=1))]

train_arr
'''

"\ntrain_arr = np.c_[input_feature_train_arr, np.array(pd.DataFrame(target_feature_train_df).reset_index().drop(['index'], axis=1))]\ntest_arr = np.c_[input_feature_test_arr, np.array(pd.DataFrame(target_feature_test_df).reset_index().drop(['index'], axis=1))]\n\ntrain_arr\n"

In [169]:
from sklearn.metrics import r2_score, classification_report, confusion_matrix, accuracy_score 

def evaluate_model(X_train,y_train,X_test,y_test,models):
    try:
        report = {}
        for i in range(len(models)):
            model = list(models.values())[i]
            # Train model
            model.fit(pd.DataFrame(X_train),y_train)

            

            # Predict Testing data
            y_test_pred =model.predict(pd.DataFrame(X_test))

            # Get R2 scores for train and test data
            #train_model_score = r2_score(ytrain,y_train_pred)
            test_model_score = accuracy_score(y_test,y_test_pred)

            report[list(models.keys())[i]] =  test_model_score

        return report

    except Exception as e:
        raise e

In [170]:
from xgboost import XGBClassifier
from sklearn.svm import SVC        # models
from sklearn.ensemble import RandomForestClassifier

def initate_model_training(train_array,test_array):
  try:
    X_train, y_train, X_test, y_test = (
    train_arr[:,:-1],
    train_arr[:,-1],
    test_arr[:,:-1],
    test_arr[:,-1]
    
)
    



    models={
    'rfc':RandomForestClassifier(),
    'xgboost':XGBClassifier(),
    'svc':SVC(),
    }
      
    model_report:dict=evaluate_model(X_train,y_train,X_test,y_test,models)
    print(model_report)
    print('\n====================================================================================\n')


    # To get best model score from dictionary 
    best_model_score = max(sorted(model_report.values()))

    best_model_name = list(model_report.keys())[
        list(model_report.values()).index(best_model_score)
    ]
    
    best_model = models[best_model_name]

    print(f'Best Model Found , Model Name : {best_model_name} , accuracy : {best_model_score}')
    print('\n====================================================================================\n')
  except Exception as e:
    raise e

initate_model_training(train_arr, test_arr)

'''' X_train, y_train, X_test, y_test = (
    #np.array(train_array.iloc[: , :-1]),
    #np.array(train_array.iloc[:,-1:]).ravel(),
    #np.array(test_array.iloc[: , :-1]),
    #np.array(test_array.iloc[:,-1:]).ravel()
    #train_array.iloc[: , :-1],
    #train_array.iloc[:,-1:],
    #test_array.iloc[: , :-1],
    #test_array.iloc[:,-1:]
    #######################
    train_array[:,:-1],
    train_array[:,-1],
    test_array[:,:-1],
    test_array[:,-1]
    #######################
    #np.column_stack([train_array[:,-1]]),
    #np.column_stack([test_array[:,-1]])         
)'''

{'rfc': 1.0, 'xgboost': 1.0, 'svc': 1.0}


Best Model Found , Model Name : rfc , accuracy : 1.0




"' X_train, y_train, X_test, y_test = (\n    #np.array(train_array.iloc[: , :-1]),\n    #np.array(train_array.iloc[:,-1:]).ravel(),\n    #np.array(test_array.iloc[: , :-1]),\n    #np.array(test_array.iloc[:,-1:]).ravel()\n    #train_array.iloc[: , :-1],\n    #train_array.iloc[:,-1:],\n    #test_array.iloc[: , :-1],\n    #test_array.iloc[:,-1:]\n    #######################\n    train_array[:,:-1],\n    train_array[:,-1],\n    test_array[:,:-1],\n    test_array[:,-1]\n    #######################\n    #np.column_stack([train_array[:,-1]]),\n    #np.column_stack([test_array[:,-1]])         \n)"

In [59]:
'''
X_train, y_train, X_test, y_test = (
    train_arr.iloc[: , :-1],
    train_arr.iloc[:,-1:],
    test_arr.iloc[: , :-1],
    test_arr.iloc[:,-1:]
)'''

'\nX_train, y_train, X_test, y_test = (\n    train_arr.iloc[: , :-1],\n    train_arr.iloc[:,-1:],\n    test_arr.iloc[: , :-1],\n    test_arr.iloc[:,-1:]\n)'

In [141]:
X_train, y_train, X_test, y_test = (
    train_arr[:,:-1],
    train_arr[:,-1],
    test_arr[:,:-1],
    test_arr[:,-1]
)


In [142]:
X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 1.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [143]:
X_train.shape

(6498, 51)

In [144]:
y_train.shape

(6498,)

In [145]:
type(y_train)

numpy.ndarray

In [146]:
y_train= np.array([y_train]).T
print(y_train)
print(y_train.shape)

[[1.]
 [1.]
 [1.]
 ...
 [0.]
 [1.]
 [1.]]
(6498, 1)


In [147]:
y_train=np.pad(y_train, ((0,0), (0,50)), mode= 'constant', constant_values=0)
y_train

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [148]:
y_train.shape

(6498, 51)

In [149]:
np.array(train_arr.iloc[:,-1:])

AttributeError: ignored

In [150]:
vect = np.array([1,1,1,1,1])
vect

array([1, 1, 1, 1, 1])

In [151]:
col_vec= np.array([vect]).T
col_vec

array([[1],
       [1],
       [1],
       [1],
       [1]])

In [152]:
col_vec_new=np.pad(col_vec, ((0,0), (0,2)), mode= 'constant', constant_values=0)
col_vec_new

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0]])